# Finding bad points
Our chains are crashing in some points because of segmentation fault. I want to identify possible bad points.

I opened the file `ownCloud/Jonathan/chauins/fluid_act/w0wa_n3/fluid_act_w0wa_n3_777632.err`.The last lines say

> `mpirun noticed that process rank 3 with PID 30920 on node sn034 exited on signal 11 (Segmentation fault)`

I then opened `ownCloud/Jonathan/chauins/fluid_act/w0wa_n3/fluid_act_w0wa_n3.4.txt`, which is the chain related to MPI process rank 3. The last recorded point in the MCMC is:

In [1]:
quantities = "weight    minuslogpost          fde_zc         log10zc         theta_i              w0      w0_plus_wa             yp2            logA              ns    theta_MC_100        omegabh2        omegach2             tau        A_planck      calib_100T      calib_217T       A_cib_217       xi_sz_cib            A_sz        ksz_norm    gal545_A_100    gal545_A_143 gal545_A_143_217    gal545_A_217    ps_A_100_100    ps_A_143_143    ps_A_143_217    ps_A_217_217              zc              As              H0          omegal          omegam        omegamh2          omegab          omegac          sigma8            s8h5      s8omegamp5     s8omegamp25               A           clamp             age           rdrag         yheused          omegan         omegan2       chi2__CMB   minuslogprior minuslogprior__0 minuslogprior__SZ            chi2 chi2__planck_2018_lowl.EE chi2__planck_2018_lowl.TT chi2__planck_2018_highl_plik.TT chi2__pyactlike.ACTPol_lite_DR4".split()
last_recorded_values = [float(x) for x in "1        504.7772     0.065492317       3.4892557       1.5434951       -2.112149      -2.4433626       1.0021302       3.0604628      0.98435725       1.0410783     0.021796498      0.12943786     0.056191636       1.0002054       1.0002271      0.99711111       28.295482      0.35050481       6.1229948      0.81191224       11.870592       8.8749365          18.7904       95.729357       178.15512       40.005396       35.846974       232.48843       3085.0036   2.1337429e-09       73.828609      0.72128967      0.27864404      0.15187949     0.039988705      0.23747174      0.86006897       1.0009697      0.45400241       0.6248787       2.1337429       1.9069296        13.17844       142.25522      0.24513867    0.0011835961    0.0006451384       672.98966       32.049014        29.963173         2.0858411       945.45637                 396.47508                 21.071161                       255.44341                       272.46671".split()]
last_recorded_point = {}
for key, value in zip(quantities, last_recorded_values):
    last_recorded_point[key] = value

In [2]:
print('Last recorded point:')
for key in ['log10zc', 'fde_zc', 'theta_i', 'w0', 'w0_plus_wa', 'theta_MC_100', 'omegabh2', 'omegach2', 'logA', 'ns', 'tau']:
    print(key+':', last_recorded_point[key])

Last recorded point:
log10zc: 3.4892557
fde_zc: 0.065492317
theta_i: 1.5434951
w0: -2.112149
w0_plus_wa: -2.4433626
theta_MC_100: 1.0410783
omegabh2: 0.021796498
omegach2: 0.12943786
logA: 3.0604628
ns: 0.98435725
tau: 0.056191636


Let's try to run this specific point:

In [1]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os, sys
import camb

# If you have many CAMB versions at your disposal, it's always useful to check which CAMB is being used.
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

Using CAMB 1.3.5 installed at /home/joaov/cosmo/CAMB/CAMBMultiFluidDE/camb


In [4]:
last_zc = 10**last_recorded_point['log10zc']
last_theta_i = last_recorded_point['theta_i']
last_fde_zc = last_recorded_point['fde_zc']
last_ns = last_recorded_point['ns']
last_w0 = last_recorded_point['w0']
last_wa = last_recorded_point['w0_plus_wa'] - last_w0
last_As = 1e-10*np.exp(last_recorded_point['logA'])
last_cosmomc_theta = 1e-2*last_recorded_point['theta_MC_100']
last_omegach2 = last_recorded_point['omegach2']
last_omegabh2 = last_recorded_point['omegabh2']
last_tau = last_recorded_point['tau']

In [14]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = last_cosmomc_theta, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = -1, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

In [15]:
print(results.Params.H0)

73.8286137421238


We found an error! It seems this cosmology cannot realize $\theta_\star = 0.0104$ for $H_0 \in [10, 100]$. Let's confirm this:

In [13]:
cosmo1 = camb.set_params(# Background
                        H0 = 10, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)
print('thetastar (H0 = 10):', results.get_derived_params()['thetastar'])
cosmo1 = camb.set_params(# Background
                        H0 = 100, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)
print('thetastar (H0 = 100):', results.get_derived_params()['thetastar'])
print('target thetastar:', 100*last_cosmomc_theta)

thetastar (H0 = 10): 0.8764759690006244
thetastar (H0 = 100): 1.0236780736885875
target thetastar: 1.0410783


Let's try another point:

In [127]:
working_values = [float(x) for x in "1       580.69389     0.036834281       3.7460373       1.0439346      -1.0272194      -1.0189491       1.0102036       3.0574109      0.97533921       1.0404165     0.022367554       0.1183387      0.06335949       1.0042431      0.99953932      0.99778482       68.246693      0.17393552       6.1650687       3.3390472       7.9023194       11.632269        24.689393       72.679448       270.17745       55.566685       20.157716        119.0703       5572.3361    2.127241e-09       71.811123      0.72582513      0.27410481      0.14135139     0.043374558      0.22947921      0.79964664      0.94363113      0.41865516      0.57859847        2.127241       1.8740593       13.451314       145.82033       0.2453979     0.001251035    0.0006451384       760.22338       32.448868        29.669463         2.7794048         1096.49                 398.18401                 22.135411                       339.90396                       336.26665".split()]
working_point = {}
for key, value in zip(quantities, working_values):
    working_point[key] = value

In [128]:
working_zc = 10**working_point['log10zc']
working_theta_i = working_point['theta_i']
working_fde_zc = working_point['fde_zc']
working_ns = working_point['ns']
working_w0 = working_point['w0']
working_wa = working_point['w0_plus_wa'] - working_w0
working_As = 1e-10*np.exp(working_point['logA'])
working_cosmomc_theta = 1e-2*working_point['theta_MC_100']
working_omegach2 = working_point['omegach2']
working_omegabh2 = working_point['omegabh2']
working_tau = working_point['tau']

In [131]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = working_cosmomc_theta, ombh2=working_omegabh2, omch2=working_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = working_w0, wa = working_wa, zc = working_zc, fde_zc = working_fde_zc, wn = 1/2, theta_i = working_theta_i,
                        # Initial Power Spectrum
                        As = working_As, ns = working_ns, AccuracyBoost=1.05, tau = working_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

It works. This could be the error! The question is: why is Cobaya not picking the `CAMBParamRangeError`?

For confirmation: let's try another crashed chain. Get the `.err` file, check the rank of the failed chain, get its last recorded point and try to run.

In [158]:
last_recorded_values = [float(x) for x in "6       506.97957      0.10731111       3.5116178       1.8272322      -2.0301903      -2.2392369       1.0007008       3.0325285      0.99056015       1.0411293     0.022281637      0.13627557      0.03691134       1.0038392      0.99905851      0.99848183       62.633333      0.68560988       6.0048908      0.49533535       8.7498779       10.466653        29.158985       102.42942        250.6759       152.15919        33.10699        95.27413       3248.0134   2.0749631e-09       76.413192      0.72728354      0.27265458      0.15920235      0.03816018      0.23338952      0.84382853      0.96531718      0.44061635      0.60975786       2.0749631       1.9273013       12.761086       138.12513       0.2453632    0.0011048828    0.0006451384        680.0167       31.068774        29.031012         2.0377621        951.8216                 398.85602                 20.644282                        260.5164                       271.80489".split()]
last_recorded_point = {}
for key, value in zip(quantities, last_recorded_values):
    last_recorded_point[key] = value

In [159]:
last_zc = 10**last_recorded_point['log10zc']
last_theta_i = last_recorded_point['theta_i']
last_fde_zc = last_recorded_point['fde_zc']
last_ns = last_recorded_point['ns']
last_w0 = last_recorded_point['w0']
last_wa = last_recorded_point['w0_plus_wa'] - last_w0
last_As = 1e-10*np.exp(last_recorded_point['logA'])
last_cosmomc_theta = 1e-2*last_recorded_point['theta_MC_100']
last_omegach2 = last_recorded_point['omegach2']
last_omegabh2 = last_recorded_point['omegabh2']
last_tau = last_recorded_point['tau']

In [160]:
print('Last recorded point:')
for key in ['log10zc', 'fde_zc', 'theta_i', 'w0', 'w0_plus_wa', 'theta_MC_100', 'omegabh2', 'omegach2', 'logA', 'ns', 'tau']:
    print(key+':', last_recorded_point[key])

Last recorded point:
log10zc: 3.5116178
fde_zc: 0.10731111
theta_i: 1.8272322
w0: -2.0301903
w0_plus_wa: -2.2392369
theta_MC_100: 1.0411293
omegabh2: 0.022281637
omegach2: 0.13627557
logA: 3.0325285
ns: 0.99056015
tau: 0.03691134


In [161]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = last_cosmomc_theta, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

CAMBParamRangeError: No solution for H0 inside of theta_H0_range

Again! Noticed: it may not match the actual rank. Let's try a third chain to confirm.

In [162]:
last_recorded_values = [float(x) for x in "2       576.73602      0.05245037       3.4026571       1.0343846      -2.1245983      -2.3215298      0.99740814       3.0269766      0.98238622       1.0415177     0.021888846      0.12324195     0.045352057       1.0019695       0.9982889      0.99817474       36.628237      0.47571244       2.5173962       1.8378346       7.3002389       14.981501         30.46687       106.67215       344.13157       136.54347       37.223424       312.15222        2527.302   2.0634751e-09       73.370657      0.72913764      0.27079524      0.14577593        0.040661      0.22893582      0.81911479      0.95627668      0.42625087       0.5908878       2.0634751       1.8845469       13.310152       144.40473      0.24518768    0.0011984173    0.0006451384       671.70662       37.061373        34.310025         2.7513489       1079.3493                 396.27718                 20.829944                       254.59949                        276.1213                 131.52138".split()]
last_recorded_point = {}
for key, value in zip(quantities, last_recorded_values):
    last_recorded_point[key] = value

In [165]:
print('Last recorded point:')
for key in ['log10zc', 'fde_zc', 'theta_i', 'w0', 'w0_plus_wa', 'theta_MC_100', 'omegabh2', 'omegach2', 'logA', 'ns', 'tau']:
    print(key+':', last_recorded_point[key])

Last recorded point:
log10zc: 3.4026571
fde_zc: 0.05245037
theta_i: 1.0343846
w0: -2.1245983
w0_plus_wa: -2.3215298
theta_MC_100: 1.0415177
omegabh2: 0.021888846
omegach2: 0.12324195
logA: 3.0269766
ns: 0.98238622
tau: 0.045352057


In [163]:
last_zc = 10**last_recorded_point['log10zc']
last_theta_i = last_recorded_point['theta_i']
last_fde_zc = last_recorded_point['fde_zc']
last_ns = last_recorded_point['ns']
last_w0 = last_recorded_point['w0']
last_wa = last_recorded_point['w0_plus_wa'] - last_w0
last_As = 1e-10*np.exp(last_recorded_point['logA'])
last_cosmomc_theta = 1e-2*last_recorded_point['theta_MC_100']
last_omegach2 = last_recorded_point['omegach2']
last_omegabh2 = last_recorded_point['omegabh2']
last_tau = last_recorded_point['tau']

In [164]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = last_cosmomc_theta, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        YHe = 0.246, WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

CAMBParamRangeError: No solution for H0 inside of theta_H0_range

## Round 2
One chain has crashed for unknown reason. No output was clear. Let's investigate the last points again

In [3]:
quantities = "weight minuslogpost fde_zc log10zc theta_i w0 w0_plus_wa yp2 logA ns theta_MC_100 omegabh2 omegach2 tau A_planck calib_100T calib_217T A_cib_217 xi_sz_cib A_sz ksz_norm gal545_A_100 gal545_A_143 gal545_A_143_217 gal545_A_217 ps_A_100_100 ps_A_143_143 ps_A_143_217 ps_A_217_217 zc As H0 omegal omegam omegamh2 omegab omegac sigma8 s8h5 s8omegamp5 s8omegamp25 A clamp age rdrag yheused omegan omegan2 chi2__CMB minuslogprior minuslogprior__0 minuslogprior__SZ chi2 chi2__planck_2018_lowl.EE chi2__planck_2018_lowl.TT".split()
last_recorded_values = [float(x) for x in "4       504.95257      0.14784456       3.5192851       1.3319669    -0.073256315      -4.8842904       1.0020333       3.0749053      0.99739003       1.0409079     0.021947889      0.14648479     0.055310195      0.99911427      0.99980689      0.99884931       56.266857      0.43206802       8.9003983       2.6650226       9.3954919       13.137496        18.003776       84.994568       182.33274       56.438229       133.63432       304.05425       3305.8651   2.1647833e-09        80.66252      0.74008229      0.25986218      0.16907782     0.033732551      0.22513809      0.91165975       1.0150721      0.46473387      0.65090642       2.1647833        1.938084       12.207522       134.24789      0.24521809   0.00099153791    0.0006451384       674.53483       34.228523        29.164095         5.0644284        941.4481                 396.32637                  20.82913                       257.37933                       266.91328".split()]
last_recorded_point = {}
for key, value in zip(quantities, last_recorded_values):
    last_recorded_point[key] = value

In [4]:
print('Last recorded point:')
for key in ['log10zc', 'fde_zc', 'theta_i', 'w0', 'w0_plus_wa', 'theta_MC_100', 'omegabh2', 'omegach2', 'logA', 'ns', 'tau']:
    print(key+':', last_recorded_point[key])

Last recorded point:
log10zc: 3.5192851
fde_zc: 0.14784456
theta_i: 1.3319669
w0: -0.073256315
w0_plus_wa: -4.8842904
theta_MC_100: 1.0409079
omegabh2: 0.021947889
omegach2: 0.14648479
logA: 3.0749053
ns: 0.99739003
tau: 0.055310195


In [5]:
print(last_recorded_point['H0'])

80.66252


In [6]:
last_zc = 10**last_recorded_point['log10zc']
last_theta_i = last_recorded_point['theta_i']
last_fde_zc = last_recorded_point['fde_zc']
last_ns = last_recorded_point['ns']
last_w0 = last_recorded_point['w0']
last_wa = last_recorded_point['w0_plus_wa'] - last_w0
last_As = 1e-10*np.exp(last_recorded_point['logA'])
last_cosmomc_theta = 1e-2*last_recorded_point['theta_MC_100']
last_omegach2 = last_recorded_point['omegach2']
last_omegabh2 = last_recorded_point['omegabh2']
last_tau = last_recorded_point['tau']

In [85]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = last_cosmomc_theta, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

In [86]:
print(results.Params.H0)

90.5270801999339


In [43]:
cosmo1 = camb.set_params(# Background
                        H0 = 80.6625, ombh2=last_omegabh2, omch2=last_omegach2,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 2, models = [2,1,0,0],
                        w0 = last_w0, wa = last_wa, zc = last_zc, fde_zc = last_fde_zc, wn = 1/2, theta_i = last_theta_i,
                        # Initial Power Spectrum
                        As = last_As, ns = last_ns, AccuracyBoost=1.05, tau = last_tau,
                        WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)

In [44]:
print(results.get_derived_params())

{'age': 12.20752309421101, 'zstar': 1093.948873520802, 'rstar': 131.6867621751439, 'thetastar': 1.0403602717939748, 'DAstar': 12.657803815217402, 'zdrag': 1061.7459817420913, 'rdrag': 134.2478995716447, 'kd': 0.15290317447725238, 'thetad': 0.16232111245407022, 'zeq': 4025.206052125477, 'keq': 0.01302384204648126, 'thetaeq': 0.7431046607241192, 'thetarseq': 0.4136732141166442}


## Part 3: The binW model

In [25]:
cosmo1 = camb.set_params(# Background
                        cosmomc_theta = 1.0423117/100, ombh2=0.022325349, omch2=0.11873374,
                        # Dark Energy Model
                        dark_energy_model = 'MultiFluidDE',
                        num_of_components = 1, models = [3,1,0,0],
                        w0 = -1.05, w1 = -2.5, w2 = -2.2, w3 = -1,
                        # Initial Power Spectrum
                        As = 2.0573112e-09, ns = 0.98087332, AccuracyBoost=1.05, tau = 0.031300839,
                        WantTransfer=False, max_l = 10000)
results = camb.get_results(cosmo1)